In [14]:
import pandas as pd
from kafka import KafkaConsumer,KafkaProducer
from time import sleep
from json import dumps,loads
from sklearn.metrics import accuracy_score
import json

In [15]:
consumer = KafkaConsumer(
        'demopc',
        bootstrap_servers = ['43.204.29.247:9092'],
        value_deserializer = lambda x: loads(x.decode('utf-8'))
)

In [16]:
import os
from joblib import load
model_path = './model/LR_model.joblib'
model = load(model_path)

In [20]:
import matplotlib.pyplot as plt
from joblib import dump
from sklearn.linear_model import LogisticRegression
i = 0
acc = []
y=[]
label = []
batch_list = []
temp = None
for x in consumer:
    if i<20:
        test_val = x.value
        c = list(test_val.values())[1:]
        feature_df = pd.DataFrame([c], columns=list(test_val.keys())[1:])
        x = feature_df.drop('Outcome', axis=1)  # X contains all columns except 'Outcome'
        l = feature_df['Outcome'].values
        a=model.predict(x)
        print(a)
        y.append(a)
        label.append(l)
        if temp is not None:
            temp = pd.concat([feature_df,temp])
        else:
            temp = feature_df
        i+=1

    else:
        feature_df = pd.DataFrame([c], columns=list(test_val.keys())[1:])
        temp = pd.concat([feature_df,temp])
        model = load(model_path)
        new_model = LogisticRegression(warm_start=True,max_iter=100)
        x, y = temp.iloc[:, :-1].astype(float), temp.iloc[:, -1]
        new_model.fit(x,y)
        joblib.dump(new_model,model_path)
        accuracy = accuracy_score(label, y)
        acc.append(accuracy)
        
        
        i=0
        
        batch_list.append(i)
        plt.bar(batch_list, acc, width=bar_width)
        plt.show()
        
        #updating the model

[0]
[0]
[1]


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0